In [2]:
# Importing importamt libraries
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
warnings.filterwarnings('ignore')

In [3]:
Bx_books=pd.read_csv("BX-Books.csv",encoding='latin1')
Bx_books_rating=pd.read_csv("BX-Book-Ratings.csv",encoding='latin1')
Bx_users=pd.read_csv("BX-Users.csv",encoding='latin1')
recomnd=pd.read_csv("Recommend.csv",encoding='latin1',header=None)

In [4]:
Bx_books.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [5]:
Bx_books_rating.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [6]:
Bx_users.head()

,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
recomnd.head()

,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [8]:
recomnd.rename(columns={0:'user_id',1:'item_id',2:'ratings',3:'timestamp'},inplace=True)

In [9]:
recomnd.head()

,user_id,item_id,ratings,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [10]:
recommend=recomnd[['user_id','item_id','timestamp']]

In [11]:
books_df=Bx_books.merge(Bx_books_rating,on='isbn',how='left')

In [12]:
books_df.head()

,isbn,book_title,book_author,year_of_publication,publisher,user_id,rating
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2.0,0.0
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8.0,5.0
2,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11400.0,0.0
3,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11676.0,8.0
4,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,41385.0,0.0


In [13]:
books_df=books_df.merge(Bx_users,on='user_id',how='left')

In [14]:
#books_df=books_df.merge(recommend,on='user_id',how='left')

In [15]:
books_df.head()

,isbn,book_title,book_author,year_of_publication,publisher,user_id,rating,Location,Age
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2.0,0.0,"stockton, california, usa",18.0
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8.0,5.0,"timmins, ontario, canada",NaN
2,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11400.0,0.0,"ottawa, ontario, canada",49.0
3,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11676.0,8.0,"n/a, n/a, n/a",NaN
4,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,41385.0,0.0,"sudbury, ontario, canada",NaN


In [16]:
books_df.shape

(954695, 9)

## Building collaberative filtering-item based

#### We will choose those users who have given at lest 100 book ratings



In [17]:
# getting the count
user=books_df.groupby('user_id')['rating'].count().reset_index()

In [18]:
threshold=200
user_id_100=user.query('rating>@threshold')
user_id_100

,user_id,rating
87,254.0,300
698,2276.0,457
862,2766.0,269
919,2977.0,227
1033,3363.0,890
...,...,...
82755,250405.0,410
82899,250764.0,222
83102,277427.0,490
83177,277639.0,265


In [19]:
# now we will select those records where userid is one of the userids of user_id_100

user_df=books_df[books_df['user_id'].isin(user_id_100.user_id)]
user_df.shape

(434834, 9)

In [20]:
# We will select those books which have been rated at least by 50 users
books=user_df.groupby('book_title')['rating'].count().reset_index()
books

,book_title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
...,...,...
149070,Ã?ngeles fugaces (Falling Angels),1
149071,Ã?Â?ber das Fernsehen.,2
149072,Ã?Â?ber die Pflicht zum Ungehorsam gegen den S...,3
149073,Ã?Â?lpiraten.,1


In [21]:
threshold=50
book_50=books.query('rating>@threshold')
book_50

,book_title,rating
464,1984,65
495,1st to Die: A Novel,147
625,2nd Chance,107
760,4 Blondes,63
1065,A Bend in the Road,106
...,...,...
147965,Year of Wonders,52
148175,You Belong To Me,54
148768,Zen and the Art of Motorcycle Maintenance: An ...,55
148901,Zoya,53


In [22]:
# now we will select those records where booktitles is one of the booktitles  of book_50

user_book_df=user_df[user_df['book_title'].isin(book_50.book_title)]
user_book_df.shape
user_book_df.columns

Index(['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher',
       'user_id', 'rating', 'Location', 'Age'],
      dtype='object')

In [23]:
# lets drop duplicates id any
user_book_df.drop_duplicates(inplace=True)

In [24]:
# We will create the sparse matrix which we can use for our reccommendation system
colab_matrix=user_book_df.pivot_table(index='book_title',columns='user_id',values='rating')
colab_matrix.fillna(0,inplace=True)
colab_matrix

user_id,254.0,2276.0,2766.0,2977.0,3363.0,4017.0,4385.0,6251.0,6323.0,6543.0,...,249111.0,249628.0,249862.0,249894.0,250184.0,250405.0,250764.0,277427.0,277639.0,278418.0
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cosine similarity approach

In [25]:
# We will calculate similarity score matrix
cos_sim=cosine_similarity(colab_matrix)
# we will prepare a Dataframe of 'colab-matrix' indeces
index_series=pd.DataFrame(colab_matrix.index)

In [26]:
lst1=[i for ind,i in enumerate(index_series['book_title']) if ind in [65,554,110,888,804]]
lst1

['Blessings', 'Deck the Halls (Holiday Classics)', 'We Were the Mulvaneys']

In [27]:
# We will define a function which will return us the top 5 books who simmillar users have evaluated
cos_sim=cosine_similarity(colab_matrix)
def recommend_movies(data,no_rec):
    rec=no_rec
    indexx=index_series[index_series.book_title==data].index
    sim_ind=pd.Series(cos_sim[indexx[0]]).sort_values(ascending=False)[1:rec+1].index
    lst1=[i for ind,i in enumerate(index_series['book_title']) if ind in sim_ind]
    return lst1
    

In [28]:
recommendations=recommend_movies('Message in a Bottle',6)
recommendations

['A Walk to Remember',
 'All I Really Need to Know',
 'Nightmares &amp; Dreamscapes',
 'Nights in Rodanthe',
 "River's End",
 'The Mulberry Tree']

### Nearest neighbour approach

In [29]:
# We will create an instance of NearestNeighbors
NN_rec=NearestNeighbors(metric='cosine')
# Fit thedata
NN_rec.fit(colab_matrix)

NearestNeighbors(metric='cosine')

In [30]:
def recommend_movies_nn(data,no_rec):
    sim_score,sim_ind_nn=NN_rec.kneighbors(colab_matrix.loc[data,:].values.reshape(1,-1),
                                           n_neighbors=no_rec+1)
    lst_nn=[i for ind,i in enumerate(index_series['book_title']) if ind in sim_ind_nn]
    lst_nn=[k for k in lst_nn if k not in data]
    return lst_nn
    

In [31]:
lst=recommend_movies_nn('Message in a Bottle',6)
lst

['A Walk to Remember',
 'All I Really Need to Know',
 'Nightmares &amp; Dreamscapes',
 'Nights in Rodanthe',
 "River's End",
 'The Mulberry Tree']

## Building collaberative filtering-User based

In [32]:
# We will see which is top rated book
books.sort_values(by='rating',ascending=False)[:5]

,book_title,rating
145432,Wild Animus,294
17985,Bridget Jones's Diary,241
121003,The Lovely Bones: A Novel,233
123508,The Notebook,214
124454,The Pelican Brief,210


In [33]:
# We will use same data which we used for Item based collaberative filtering-user_book_df
# We will prepare a matrix out of it for our user based reccomendation system
user_based=user_book_df.pivot_table(index='user_id',columns='book_title',values='rating')

user_based

book_title,1984,1st to Die: A Novel,2nd Chance,4 Blondes,A Bend in the Road,A Case of Need,"A Child Called \It\"": One Child's Courage to Survive""",A Civil Action,A Day Late and a Dollar Short,A Fine Balance,...,Winter Moon,Winter Solstice,Wish You Well,Without Remorse,Wuthering Heights,Year of Wonders,You Belong To Me,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zoya,"\O\"" Is for Outlaw"""
user_id,,,,,,,,,,,,,,,,,,,,,
254.0,9.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2276.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2766.0,NaN,NaN,NaN,NaN,7.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2977.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN
3363.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250405.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250764.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
277427.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
 #We will subtract the mean from each item as some user are open but some are stric so ratings given by them may differ
user_norm=user_based.subtract(user_based.mean(axis=1),axis=0)


In [73]:
user_norm.tail()

book_title,1984,1st to Die: A Novel,2nd Chance,4 Blondes,A Bend in the Road,A Case of Need,"A Child Called \It\"": One Child's Courage to Survive""",A Civil Action,A Day Late and a Dollar Short,A Fine Balance,...,Winter Moon,Winter Solstice,Wish You Well,Without Remorse,Wuthering Heights,Year of Wonders,You Belong To Me,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zoya,"\O\"" Is for Outlaw"""
user_id,,,,,,,,,,,,,,,,,,,,,
250405.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250764.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.826087,NaN
277427.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277639.0,NaN,NaN,-2.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278418.0,NaN,NaN,NaN,NaN,NaN,-0.344262,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# We will calculate user similarity score by using cosine similarity
user_sm=cosine_similarity(user_norm.fillna(0))

In [80]:
def recomend_books(target_user,no_users,no_books):
    indexx_user=index_series_user[index_series_user.user_id==target_user].index
    sim_ind=pd.Series(user_sm[indexx_user[0]]).sort_values(ascending=False)[0:no_users].index
    user_lst=[i for ind,i in enumerate(index_series_user.user_id) if ind in sim_ind]
    filtered_data=user_norm[user_norm.index.isin(user_lst)]
    target_data=user_norm[user_norm.index==target_user].dropna(axis=1,how='all')
    # selecting those books which are not read by target user
    lst=[i for i in filtered_data.columns if i not in target_data.columns] 
    filtered_data=filtered_data[lst]
    filtered_data_t=filtered_data.transpose()
    filtered_data_t.fillna(0,inplace=True)
    filtered_data_t['avg']=filtered_data_t.mean(axis=1)
     # getting tptal ratings for the books
    filtered_data_t['rating']=books[books['book_title'].isin(filtered_data_t.index)]['rating'].values
    filtered_data_t['weighted_rating']=((filtered_data_t['avg']*95)+(5*filtered_data_t['rating']))/100
    filtered_data_t=filtered_data_t.sort_values(by='weighted_rating',ascending=False)
    return list(filtered_data_t.index[0:no_books])

In [81]:
books_recomend=recomend_books(2276,10,5)
books_recomend

['Wild Animus',
 "Bridget Jones's Diary",
 'The Lovely Bones: A Novel',
 'The Notebook',
 'The Pelican Brief']

In [83]:
books_recomend=recomend_books(250405,10,5)
books_recomend

["Bridget Jones's Diary",
 'The Lovely Bones: A Novel',
 'The Notebook',
 'The Pelican Brief',
 'The Firm']